### Tif_read_write - script to check ability to read and save a tif

Bottom line: no way to use xarray to write a geotiff. This describes how to process a raster and get it back into a DataArray.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
#from dask.distributed import LocalCluster
from scipy import interpolate, signal
%matplotlib inline

In [2]:
fdir = "C:/crs/proj/2019_DorianOBX/WayneWright_flights/"
fname = ("2019-08-30_1m_DEM_v1_crop.tif")

In [3]:
fn = fdir+fname
print(fn)

# open the tif with XArray as a DataArray
da = xr.open_rasterio(fn)
da

C:/crs/proj/2019_DorianOBX/WayneWright_flights/2019-08-30_1m_DEM_v1_crop.tif


<xarray.DataArray (band: 1, y: 22393, x: 25165)>
[563519845 values with dtype=float32]
Coordinates:
  * band     (band) int32 1
  * y        (y) float64 3.88e+06 3.88e+06 3.88e+06 ... 3.858e+06 3.858e+06
  * x        (x) float64 3.801e+05 3.801e+05 3.801e+05 ... 4.052e+05 4.052e+05
Attributes:
    transform:      (1.0, 0.0, 380059.3422399781, 0.0, -1.0, 3880457.7811244014)
    crs:            +init=epsg:3725
    res:            (1.0, 1.0)
    is_tiled:       0
    nodatavals:     (-32767.0,)
    scales:         (1.0,)
    offsets:        (0.0,)
    units:          ('metre',)
    AREA_OR_POINT:  Point

In [4]:
# The dataArray has a singleton value
np.shape(da.values)

(1, 22393, 25165)

In [5]:
%%time
# smooth with 2D running median
zs = signal.medfilt2d(np.squeeze(da.values), kernel_size=11)

Wall time: 1min 13s


In [6]:
# Simplest way to build new dataArray is to copy the one we loaded
da2=da.copy()
da2

<xarray.DataArray (band: 1, y: 22393, x: 25165)>
array([[[-32767., -32767., -32767., ..., -32767., -32767., -32767.],
        [-32767., -32767., -32767., ..., -32767., -32767., -32767.],
        [-32767., -32767., -32767., ..., -32767., -32767., -32767.],
        ...,
        [-32767., -32767., -32767., ..., -32767., -32767., -32767.],
        [-32767., -32767., -32767., ..., -32767., -32767., -32767.],
        [-32767., -32767., -32767., ..., -32767., -32767., -32767.]]],
      dtype=float32)
Coordinates:
  * band     (band) int32 1
  * y        (y) float64 3.88e+06 3.88e+06 3.88e+06 ... 3.858e+06 3.858e+06
  * x        (x) float64 3.801e+05 3.801e+05 3.801e+05 ... 4.052e+05 4.052e+05
Attributes:
    transform:      (1.0, 0.0, 380059.3422399781, 0.0, -1.0, 3880457.7811244014)
    crs:            +init=epsg:3725
    res:            (1.0, 1.0)
    is_tiled:       0
    nodatavals:     (-32767.0,)
    scales:         (1.0,)
    offsets:        (0.0,)
    units:          ('metre',)
    AREA_OR_POINT:  Point

In [7]:
# Have to add the singleton dimension when replaceing the data
da2.values[0,:,:]=zs